# Image Classification using CNNs in Keras - Project.


# Contents:
------------------------
-  . <a href = #link100>Context:</a>
- 1. <a href = #link1>Importing Libraries</a>
- 2. <a href = #link2>Reading and Review of the dataset.</a>
- 3. <a href = #link3>Transposing index and columns</a>
- 4. <a href = #link4>EDA Discriptive Observations</a>
- 5. <a href = #link5> Dataset splitting Train and Test.</a>
- 6. <a href = #link6>One Hot encoding to target values..</a>
- 7. <a href = #link7>Gussian Blurring.</a>
- 8. <a href = #link8>Building the CNN a Model.</a>
- 9. <a href = #link9>Model Evaluation.</a>
- 10. <a href = #link10>Model Retraing.</a>
- 11. <a href = #link11>Conclusion.</a>
- 12. <a href = #link12>References & GitHub Link.</a>
    

# <a id='link100'> Context:</a>

# Data Description:

You are provided with a dataset of images of plant seedlings at various stages of grown. Each image has a filename that is its unique id. The dataset comprises 12 plant species. The goal of the project is to create a classifier capable of determining a plant's species from a photo.Dataset:
 
**The dataset can be download from Olympus.**
The data file names are:

* images.npy
* Label.csv

The original files are from Kaggle. Due to the large volume of data, the images were converted to images.npy file and the labels are also put into the Labels.csv.



Can you differentiate a weed from a crop seedling?
The ability to do so effectively can mean better crop yields and better stewardship of the environment.The Aarhus University Signal Processing group, in collaboration with University of Southern Denmark, has recently released a dataset containing images of unique plants belonging to 12 species at several growth stages.

**Objective:**

To implement the techniques learnt as a part of the course.

**Learning Outcomes:**
*  Pre-processing of image data.
*  Visualization of images.
*  Building CNN.
*  Evaluate the Model.
*  The motive of the project is to make the learners capable to handle images/image classification  problems, during this process you should also be capable to handle real image files, not just limited  to a numpy array of image pixels.

**Guide to solve the project seamlessly:**
Here are the points which will help you to solve the problem efficiently:
* Read the problem statement carefully from start to end (including the note at the end). The   highlighted part in the attached problem statement should not be missed.
* Download the dataset from the Olympus platform.
* Upload the "images.npy" and “Labels.csv” file to google drive.
* Then you can use the dataset path in the Google Colab notebook to do further steps related to project problem statement.
* You can set runtime type to “GPU” in Google Colab, so that the code will run faster as you will be using CNN to fit your model.

**Steps and tasks:**
1. Import the libraries, load dataset, print shape of data, visualize the images in dataset.
2. Data Pre-processing:

a. Normalization.
b. Gaussian Blurring.
c. Visualize data after pre-processing.

3. Make data compatible: 
* a-Convert labels to one-hot-vectors.
* b-Print the label for y_train[0].
* c-Split the dataset into training, testing, and validation set.

(Hint: First split images and labels into training and testing set with test_size = 0.3. Then further split test data into test and validation set with test_size = 0.5)
* d- Check the shape of data, Reshape data into shapes compatible with Keras models if it’s not already. If it’s already in the compatible shape, then comment in the notebook that it’s already in compatible shape.

4. Building CNN: 
* a. Define layers.
* b. Set optimizer and loss function. (Use Adam optimizer and categorical crossentropy.)
5. Fit and evaluate model and print confusion matrix. (10 Marks)
6. Visualize predictions for x_test[2], x_test[3], x_test[33], x_test[36], x_test[59]. (5 Marks)


# Environment and Algorithms techniques.
* cv2
* numpy 
* pandas as pd
* seaborn as sns
* matplotlib import pyplot as plt
* tensorflow as tf
* sklearn.model_selection import train_test_split
* keras
* keras.models/Sequential
* keras.layers/Dense
* sklearn.metrics/accuracy_score
* sklearn.metrics/confusion_matrix
* sklearn.metrics/classification_report

# <a id='link1'> Importing Libraries</a>

In [ ]:
# The following line od codes to import necessary libraries.
import numpy as np
import pandas as pd
import math 
import cv2
import os

from glob import  glob
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense, 
    Dropout, 
    Flatten, 
    Conv2D, 
    MaxPooling2D, 
    MaxPool2D,
    GlobalMaxPooling2D,
    BatchNormalization
)

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix

# <a id='link2'> Reading and Review of the dataset.</a>

## Loading the data.
* We use panda's read_csv to read train.csv into a dataframe.
* Then we separate our images and labels for supervised learning.
* We also do a train_test_split to break our data into two sets, one for training and one for testing. This let's us measure how well our model was trained by later inputting some known test data.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Set the path to the dataset folder. (The dataset contains image folder: "train")
trainLabel = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Labels.csv').copy()

In [ ]:
trainLabel.head(10)

In [ ]:
# The following line of code to Load images file.
trainImg= np.load('/content/drive/My Drive/Colab Notebooks/images.npy')

In [ ]:
trainImg

# <a id='link3'>Transposing index and columns.</a>

In [ ]:
# The following line of codes to print array shape.
print(f'Training image array shape:{trainImg.shape}')
print(f'Training target labels:{trainLabel.shape}')

In [ ]:
trainLabel.describe()

# <a id='link4'> EDA Discriptive Observations.</a>

In [ ]:
# The following line of code is to randomize the order of training set.
SEED = 42
train_df = trainLabel.sample(frac=1, random_state=SEED)
train_df.index = np.arange(len(trainLabel)) # This to reset indices
train_df.head(15)

In [ ]:
plt.hist(train_df['Label'])
plt.title('Frequency Histogram of Species')
plt.figure(figsize=(20,18))
plt.show()

# Checking the dataset for error, duplicates.

In [ ]:
trainLabel.isnull().sum()

In [ ]:
pd.isnull(trainLabel).count()

In [ ]:
trainLabel.value_counts()

In [ ]:
# The following code to check duplicates in the datasets.
dupes = trainLabel.duplicated()
sum(dupes)

# Pre-Processing.

* Normalizing the data.
* Tain image and testing image needs to be normalized to 0-1 by dividing the values by 255.

In [ ]:
trainImg = trainImg.astype('float32')
trainImg /=255
# the following line of codes to check the nomalized data.
print(f'Shape of the Train array:{trainImg.shape}')
print(f'Minimum value in the Train Array:{trainImg.min()}')
print(f'Maximum value in the Train Array:{trainImg.max()}')


# <a id='link5'>Spliting The Dataset.</a>
## Spliting the dataset into training, testing, and validation set

In [ ]:
# Step 1:
# The following line of codes to split the xtrain dataset.
x_train, x_test, y_train, y_test = train_test_split(trainImg, trainLabel, test_size=0.3, random_state=42)
x_train.shape, x_test.shape

In [ ]:
# Step 2:
# The following line of codes to split the validation dataset.
x_test, x_validation, y_test, y_validation = train_test_split(x_test, y_test, test_size=0.5, random_state=42)

x_test.shape, x_validation.shape

# <a id='link6'>One Hot encoding to target values.</a>

In [ ]:
from sklearn.preprocessing import  LabelBinarizer
encoder = LabelBinarizer()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)
y_validation = encoder.fit_transform(y_validation)

## Converting Target Variables.

* Will convert the target variable to one-hot.
* Will Convert String categorical to numeric.
* Will use tensorflow.keras.utils.to_categorical to convert to binary array.

In [ ]:
# The following line of codes to Print the label for y_train[0] target variable.
y_train[0]

In [ ]:
y_train

In [ ]:
print(y_train.shape)
print(y_test.shape)

# <a id='link7'>Gussian Blurring.</a>
Image blurring by convoliving the image with a low-pass filter kernel to be able to remove the noise also to remove high frequency content e.g (noise, edges) from the image resulting in edges being blurred when the filter is aplied.

In [ ]:
# the following line of codes to preview the image before Gusian blur.
plt.imshow(x_train[1], cmap='gray')

In [ ]:
# Preview the image before Gaussian Blur
plt.imshow(x_train[1], cmap='gray')

In [ ]:

plt.imshow(cv2.GaussianBlur(x_train[1], (15,15), 0))

In [ ]:
# the following line of codes to apply the gaussian blur to each 128x128 pixels array (image)
# to reduce the noise in the the image.
for idx, img in enumerate(x_train):
  x_train[idx] = cv2.GaussianBlur(img, (5, 5), 0)

In [ ]:
# The following line of codes to preview the image after applying the Gaussian Blur.
plt.imshow(x_train[0], cmap='gray')

In [ ]:
# The following line of codes to apply Gaussian Blue to test and validation.
for idx, img in enumerate(x_test):
  x_test[idx] = cv2.GaussianBlur(img, (5, 5), 0)

In [ ]:
for idx, img in enumerate(x_validation):
  x_validation[idx] = cv2.GaussianBlur(img, (5, 5), 0)

# <a id='link8'>Building the CNN a Model.</a>

# Steps:
* Intializing the CNN Classifier
* Adding Convolution layer with 32 kernels of 3x3 shape.
* Adding Maxpooling layer of size 2x2.
* Flatten will be the inpue array.
* Adding dense layer with relu activation function.
* Dropout the probability.
* Adding softmax dense layer as output.

In [ ]:
# The following line of codes to build the CNN model.
def create_model(input_shape, num_classes):
  # Initializing the CNN Classifer.
  model = Sequential()

  # Adding the convolution layer with 32 filters and 3 kernels.
  model.add(Conv2D(32, (3,3), input_shape=input_shape, padding='same', activation=tf.nn.relu))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(rate=0.25))

  # Adding convolution layer with 32 filters and 3 kernels.
  model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation=tf.nn.relu))
  model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation=tf.nn.relu))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(rate=0.25))

  # Adding the Convolution layer with 32 filter and 3 kernels.
  model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation=tf.nn.relu))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(rate=0.25))

  # Flatten the 2D array to 1D array.
  model.add(Flatten())

  # Creating a fully connected layers with 512 units.
  model.add(Dense(512, activation=tf.nn.relu))
  model.add(Dropout(0.5))

  # Adding a fully connected layer with 128 neurons.
  model.add(Dense(units = 128, activation = tf.nn.relu )) 
  model.add(Dropout(0.5))

  # The final output layer with 12 neurons to predict the categorical classification.
  model.add(Dense(units= num_classes, activation= tf.nn.softmax))
  return model

**Model includes.**
* **The Sequential** defines a sequence of layers.
* **Conv2D:** Keras Conv2D is a 2D convolution layer, this creates a convolution kernel that is is wind with layers input which helps preduce a tensor of outputs.
* **MaxPool2D:** The objective is to down-sample an input representation. 
* **Flatten:** Convert the 2D to 1D array. 
* **Dense:** adds a layers of neurons.

* **Activation Functions:** 
* **Relu:** Relu effectively means "if X>0return X, else return 0' -- so what it does it only passes values 0 or greater to the next layer in the network.
* **Sofmax:** function is also a type of sigmoid function but is handy when we are trying to handle classification problems.
* Usually used when trying to handle multiple classes. The softmax function would squeeze the outputs for each class between 0 and 1 and would also divide by the sum of the outputs.

In [ ]:
class mycallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.95):
      print('\nReached 95% accuracy so cancelling training!')
      self.model.stop_training = True

callbacks = mycallback()

es = EarlyStopping(monitor='val_accuracy', mode='min', verbose=1, patience=10)


In [ ]:
input_shape = x_train.shape[1:] #Input shape of xtrain
num_classes = y_train.shape[1] # Target ccolumn size

model = create_model(input_shape, num_classes)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) # Optimizer

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(x_train, y_train, validation_data=(x_validation, y_validation), epochs=30, batch_size=100, callbacks=[callbacks])

* The above score shows that the accuracy is good, as we used number of epochs = 30. If we use more epochs and tune the hyper-parameters more then we can get some more accuracy score. 

In [ ]:
# Plotting the history.
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('Epoch', fontsize=18)
plt.ylabel(r'Loss', fontsize=18)
plt.legend(('loss train','loss validation'), loc=0)

In [ ]:
# The following line of codes to plot the accuracy.
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('Epoch', fontsize=18)
plt.ylabel(r'Loss', fontsize=18)
plt.legend(('accuracy train', 'accuracy validation'), loc=0)

# <a id='link9'>Model Evaluation.</a>

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print('Test loss: {:.2f} \n Test accuracy: {:.2f}'.format(loss, accuracy))

loss, accuracy = model.evaluate(x_train, y_train)
print('Train lodd: {:.2f} \n Train accuracy: {:.2f}'.format(loss, accuracy))

* Model is overfitting since training accuracy is 97% and testing accuracy is 83%. let's stop it before 18 epoch

# <a id='link10'>Model Retraing.</a>

In [ ]:
# The following line of codes to rint the summary report.
model = create_model(input_shape, num_classes)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) # Optimizer

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(x_train, y_train, validation_data=(x_validation, y_validation), epochs=18, callbacks=[callbacks])

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)
print('Test loss: {:.2f} \n Test accuracy: {:.2f}'.format(loss, accuracy))

loss, accuracy = model.evaluate(x_train, y_train)
print('Train loss: {:2f} \n Train accuracy: {:.2f}'.format(loss, accuracy))


* The early stopping helping model to balance accuracy b/w test and training.

# Saving the Model

In [ ]:
model.save('./CIFAR_classifier.h5')                     # save classifier (model) and architecture to single file

model.save_weights('./CIFAR_classifier_weights.h5')     # weights are saved directly from the model.

## Classification Report and Confusion matrix Plotting:
Following line of code to rint a classification report and Confusion to measure the quality of predictions from a classification algorithm. it will indicate how many predictions are True and how many are not. it will point the True positives, and False positives, True negatives and false negatives.

In [ ]:
# The following line of codes to rint the classification report.
from sklearn.metrics import classification_report
y_pred = model.predict(x_test)
print(classification_report(y_test.astype('int'), y_pred.astype('int')))

In [ ]:
y_pred = model.predict(x_test)
y_pred = (y_pred > 0.5)

In [ ]:
print('======== Classification Matrix ========')
print(confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

In [ ]:
print('=========== Classification Report ====================')
print(classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1)))

## Observation.

* **Precision:** Out of all the positive classes we have predicted correctly, how many are actually positive.

* **Recall:** Out of all the positive classes, how much we predicted correctly. It should be high as possible.

* **F1-Score:** F1 Score is the weighted average of Precision and Recall. Therefore, this score takes both false positives and false negatives into account. Intuitively it is not as easy to understand as accuracy, but F1 is usually more useful than accuracy, especially if you have an uneven class distribution


## Visualize Predictions.

In [ ]:
y_pred = encoder.inverse_transform(y_pred)

index = 2
plt.imshow(x_test[index], cmap='gray')
print('Predicted Label:', y_pred[index])

In [ ]:
index = 3
plt.imshow(x_test[index], cmap='gray')
print('Predicted Label:', y_pred[index])

In [ ]:
index = 33
plt.imshow(x_test[index], cmap='gray')
print('Predicted Label:', y_pred[index])

In [ ]:
index = 36
plt.imshow(x_test[index], cmap='gray')
print('Predicted Lable:', y_pred[index])

In [ ]:
index = 59
plt.imshow(x_test[index], cmap='gray')
print('Predicted Label:', y_pred[index])

# <a id='link11'>Conclusion.</a>

The purpose of this case study is to make a prediction to differentiate a weed from a crop seeding. the ability to do so effectively can mean better crop yields and better stewardship of the environment. 
The Aarhus university Sigal processing group, in collaboration with the University of Southern Denmark, has recently released a dataset containing images of unique plants belonging to 12 species at several growth stages.

We presented a deep learning-based approach for plant image prediction which accurately predicted to differentiate a weed from crop seeding yield the entire cord belt.
Most importantly, our methodology moved beyond prediction as it provided key results towards explaining yield prediction variable importance by time period.

By implementing different CNN deep machine learning techniques. The solution by building a CNN model that has ab accuracy of approximately from 80% to 85% prediction capability with a high percentage of accuracy also more improvements can be made to more improve it.

by visualizing the results in order to check which class has the best and worst performance and further necessary parameters tunning steps can be taken in order to improve the results, seek the accuracy rate in order to find the validation accuracies and losses converge nicely.

The model F1 score results within the validation accuracy of 84% and an error loss rate of approximately 13%.

In this case study we discovered how to create deep CNNs in Keras for image classification. After working through this case study we learned:

* About the CIFAR-10 dataset and how to load it in Keras and plot examples from the dataset.
* How to train and evaluate a Convolutional Neural Network on the problem.

# <a id='link12'>References & GitHub Link..</a>

* https://github.com/command20/This-repository-records-all-the-project-works-that-I-did-in-my-PGP-AIML-program-with-UT-Austin-and-G


* https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/
* https://machinelearningmastery.com/difference-between-a-batch-and-an-epoch/
* https://keras.io/api/callbacks/
* https://towardsdatascience.com/3-ways-to-build-neural-networks-in-tensorflow-with-the-keras-api-80e92d3b5b7e
* https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
* https://www.tensorflow.org/guide/function#python_or_tensor_args
* https://www.tensorflow.org/api_docs/python/tf/function
* https://developer.nvidia.com/cuda-gpus
* https://towardsdatascience.com/tensorflow-gpu-installation-made-easy-use-conda-instead-of-pip-52e5249374bc
* https://www.tensorflow.org/tutorials/images/classification
* Xing E., Jordan M., Karp R, “Feature selection for high￾dimensional genomic microarray data,” Proceedings of the Eighteenth International Conference on Machine Learning, Massachusetts, USA: Morgan Kaufmann, 2001, 601–608.
